In [ ]:
#Parte comum:

import pandas as pd
import geopandas as gpd
# from jupyterthemes import jtplot
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.subplots as ps
import basedosdados as bd


states = bd.read_sql(
    '''
    SELECT DISTINCT id_municipio AS id, sigla_uf
    FROM  basedosdados.br_bd_diretorios_brasil.setor_censitario AS censo
    GROUP BY id_municipio, sigla_uf
    ''',
    billing_project_id='bd-geo-covid')

cities = gpd.read_file("geojson\\brasil.json", encoding='utf-8')

brasil_data = pd.merge(states, cities, on='id')
brasil_data = gpd.GeoDataFrame(brasil_data, geometry='geometry')

RJ_data = brasil_data[brasil_data["sigla_uf"] == "RJ"].reset_index(drop=True)
RJ_data = RJ_data.astype({"id": int})

covid = pd.read_csv('caso_full.csv.gz', compression = 'gzip')
covid.rename(columns={"city_ibge_code": "id", "last_available_confirmed_per_100k_inhabitants": "cases_per_100k", "last_available_death_rate": "death_rate"}, inplace=True)
covid = covid[covid["id"] > 1000].astype({"id": int})
covid_rj = covid[(covid["state"] == "RJ") & (covid["date"] == "2021-11-17")].reset_index(drop=True)

covid_rj = covid_rj[["id", "estimated_population", "cases_per_100k", "death_rate"]]

RJ_fulldata = pd.merge(RJ_data, covid_rj, on='id')


# Análise de correlação entre indicadores sociais e Covid-19 no Estado do Rio de Janeiro

Nessa análise buscaremos responder à dúvida quanto à existência ou não de uma ligação entre o IDH e índices relacionados e a incidência da pandemia de Covid-19 no estado do Rio de Janeiro. Os dados aqui utilizados estão disponíveis na [Base dos Dados](https://basedosdados.org/blog/2), de fonte principal do IBGE, no próprio site do [IBGE](https://www.ibge.gov.br/) e, no caso dos dados relacionados ao Covid, do site [Brasil.io](https://brasil.io/dataset/covid19/caso_full/). Estes últimos são considerados quanto ao aspecto acumulado ao longo da pandemia, tendo sido extraídos no início da elaboração do trabalho, em 17/11/2021.

A escolha de realizar a análise quanto o estado do Rio de Janeiro se deu por os 3 integrantes do grupo serem fluminenses, de diferentes cidades, e nos interessava a possibilidade de fazer comparações.

## IDH 

Iniciaremos a análise falando do próprio IDH, Índice de Desenvolvimento Humano, que é bastante conhecido, e leva em conta aspectos de expectativa de vida, educação e PIB per capita, disponibilizado quanto a cada município do país pelo IBGE. A última atualização deste foi no censo de 2010.

 Começamos lendo o `idh.csv` e realizando uma junção com o `RJ_data` através dos nomes de municípios, já que o IBGE não disponibilizava os ids. Veja abaixo uma parcela do database:

In [ ]:
idh = pd.read_csv('idh.csv')
RJ_idh = pd.merge(RJ_data, idh, on='name')
RJ_idh = RJ_idh.drop(['description'], axis=1)

RJ_idh.head()

Agora que já temos os dados que nos interessam unificados, podemos iniciar a plotagem, utilizando os dados de geometria e através do `plotly express`. É possível observar que cidades da Região Metropolitana do estado possuem altos IDHs, como Niterói, enquanto a Região Serrana possui valores baixos, como por exemplo em Sumidouro. Veja abaixo a plotagem, em que é possível passar o mouse para ver informações:

In [ ]:
fig = px.choropleth(RJ_idh,
                   geojson = RJ_idh["geometry"],
                   locations = RJ_idh.index,
                   color = "IDH",
                   hover_name = "name",
                   color_continuous_scale = "Purples"
)

fig.update_geos(
                projection=dict(
                    scale=55
                    ),
                center = dict(
                    lat=-22.208333,
                    lon=-42.896388
                    )
)

fig.update_layout(
    title_text = "IDH de 2010, por município do RJ",
    margin={"r":0,"t":50,"l":0,"b":50}
)

fig.show()

Análise da relação entre índices de educação e incidência do Covid-19 no estado do Rio de Janeiro

Temos interesse em realizar uma análise comparativa, então faremos a seguir um scatter plot relacionando a quantidade de mortes por Covid e o IDH, para podermos descobrir se há ou não uma correlação explícita. A escolha de fazer essa observação a partir das mortes, e não pela incidência da doença (número de casos) se dá pela quantidade de fatores que geram variabilidade nesta segunda, tornando a relação menos direta.

Por exemplo, municípios que investiram em testagem em massa certamente têm o número de casos inflados em relação a outros que testaram menos a população, não refletindo puramente a realidade. Analisando os óbitos, certamente nos deparamos com casos não notificados de morte por Covid, mas essa variação tende a ser melhor distibuída pelo estado, já que teoricamente depende menos de fatores externos

In [ ]:
covid_idh = pd.merge(RJ_idh, covid_rj, on='id')

fig = px.scatter(covid_idh, x="death_rate", y="IDH", hover_data=['name'], trendline="ols", 
color_discrete_sequence=['#734CB3'],trendline_color_override="orange",
 title="Gráfico de dispersão entre taxa de mortalidade e IDH",
 labels={"death_rate": "Taxa de mortalidade",
        "name": "Município"})
fig.show()

Observe que o gráfico nos fornece uma distribuição desigual e aparentemente desordenada, que nos leva a acreditar que não há uma correlação direta entre os fatores. Niterói e Sumidouro, por exemplo, com respectivamente maior e menor IDH, não têm uma taxa de morte pela doença tão diferente. Traçamos uma reta de tendência, buscando analisar a disposição dos pontos e algum tipo de correlação entre os dados, mas a distância dos pontos à reta era tal que encontramos um baixíssimo $R^2$ (que pode ser observado passando o mouse pela reta, assim como para informações nos pontos).

Essa comparação nos indica que não há correlação clara entre o IDH de cada município e a incidência da pandemia de Covid-19. No entanto, conforme observamos anteriormente, esse índice é formado por 3 indicadores, e buscaremos analisar cada um destes individualmente:

## Educação

Para a análise quanto à relação do Covid com índices educacionais de cada município fluminense, utilizaremos o IDEB, Índice de Desenvolvimento da Educação Básica, utilizado para medir o nível das escolas públicas brasileiras em uma escala de 0 a 10 levando em conta o desempenho escolar formal dos estudantes e o chamado fluxo escolar (que leva em conta repetências, evasão escolar, etc.). Essa base é obtida através da Base dos Dados, e realizamos a seguir a obtenção dos dados:

In [ ]:
ideb = bd.read_sql(
    '''
    SELECT id_municipio AS id, sigla_uf, AVG(ideb) AS ideb , ano
    FROM  basedosdados.br_inep_ideb.municipio
    WHERE ano = 2019 AND rede = 'publica' 
    GROUP BY id_municipio, sigla_uf, ano
    ''',
    billing_project_id='bd-geo-covid')


Realizamos o mesmo processo anterior, filtrando as informações importantes e filtrando o que é necessário (o id maior que 1000, por exemplo, retira ids incorretos, como 0). Fazemos novamente o `merge` com RJ_data, mas dessa vez podemos utilizar o id, obetendo os dados de geometria. 

In [ ]:
ideb_data =  ideb.astype({"id": int})
ideb_data = ideb_data[ideb_data["id"] > 1000]
ideb_data = ideb_data[(ideb_data["sigla_uf"] == "RJ")].reset_index(drop=True)

RJ_ideb = pd.merge(RJ_data, ideb_data, on='id')
RJ_ideb = RJ_ideb.drop(['description'], axis=1)


RJ_ideb.head()



Podemos realizar a plotagem geométrica. Note como a distribuição é diferente do próprio IDH, já que a Região Metropolitana, por exemplo, que possuía os valores maiores, não tem um IDEB muito alto. Os melhores índices, na verdade, se concentram na Região Noroeste do estado.

In [ ]:
fig = px.choropleth(RJ_ideb,
                   geojson = RJ_ideb["geometry"],
                   locations = RJ_ideb.index,
                   color = "ideb",
                   hover_name = "name",
                   color_continuous_scale = "Blues",
                   labels = {
                       "ideb": "IDEB ",
                   },
)

fig.update_geos(
                projection=dict(
                    scale=55
                    ),
                center = dict(
                    lat=-22.208333,
                    lon=-42.896388
                    )
)

fig.update_layout(
    title_text = "IDEB de 2019, por município do RJ",
    margin={"r":0,"t":50,"l":0,"b":50}
)

fig.show()


Agora que observamos a diferença de distribuição, temos interesse em analisar se há, ou não, correlação a partir desse índice. Faremos novamente o scatter plot e a linha de tendência, analisando o $R^2$ para observar a própria correlação.

In [ ]:
covid_ideb = pd.merge(RJ_ideb, covid_rj, on='id')

fig = px.scatter(covid_ideb, x="death_rate", y="ideb" , hover_data=['name'], 
trendline="ols",trendline_color_override="orange",
title="Gráfico de dispersão entre taxa de mortalidade e IDEB",
labels={"death_rate": "Taxa de mortalidade",
        "name": "Município",
        "ideb":"IDEB"})
fig.show()

O resultado é melhor que o anterior. Visualmente, é possível perceber algum nível de relação entre as variáveis, uma que que o IDEB é menor, a taxa de mortalidade tende a ser menos também. Já o próprio $R^2$ nos indica que essa correlação não pode ser formalmente induzida. O valor, que varia entre 0 e 1, tem uma aumento considerável quando comparado à análise do IDH, mas ainda não é suficiente para podermos dizer que a correlação existe.